In [27]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 10:07:23 2021
"""

###############Important Note ##################
# Code created by Edwin Goh, Jaejin Lee, Su Qin
# Version 0.2
# For enqueries, please send email to Eurex Asia Sales Support <Eurex_Asia_Sales_Support@eurexchange.com>

###############Version History ##################
# Version 0.1: Input files, function and code for even pages by Edwin
# Version 0.2: checkSever() function, Fixed overlapping header
# Version 0.3: Fixed minor bugs on the format
# Version 0.4: Added 'handle' so that adding one month data to existing excel file
# Version 0.5: Separated extracting whole/month data function, added dependencies
# Version 0.6: Merged Volume data

############### Library Summary ##################
# requests: to establish connection with url site
# urllib: to encode string to be used in a query part of a url
# bs4: For importing html element
# datetime: Calculate execution time
# pandas: for dataframe manipulation, exporting excel output file
# sys: for termination of the program
# openpyxl: for loading and saving the excel file

import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import sys
from pandas import ExcelWriter
from openpyxl import load_workbook
import copy

###############User Input ##################
handle = '202102'  # YYYYMM#
choice = 0         # 0: Adding to existing, 1: Extracting whole data

url_1 = 'https://www.sitca.org.tw/ROC/Industry/IN2608.aspx?pid=IN22603_01'
url_2 = 'https://www.sitca.org.tw/ROC/Industry/IN2610.aspx?pid=IN22603_03'

proxies = {
    'http': 'http://webproxy.deutsche-boerse.de:8080/',
    'https': 'http://webproxy.deutsche-boerse.de:8080/',
}

############### Functions ##################
def checkServer(url, name):
    # Function checks if the url server is up or down and return the 'soup' if the server is up
    # Upward dependencies: None
    # Downward dependedncies: None
    response = requests.get(url)

    if response.status_code != requests.codes.ok:
        sys.exit('Error connecting the server. Please check if the server is up')
    else:
        print(f'{name} Server is up. Continue on scraping...')
        return BeautifulSoup(response.text, 'html.parser')


def getMonthData(month, url, soup, name):
    # Function returns the dataframe of parameter 'month'
    # Upward dependencies: checkServer
    # Downward dependedncies: None
    
    #Extracting the table
    VIEWSTATE = soup.find("input", {"id": "__VIEWSTATE"}).attrs['value']
    VIEWSTATEGENERATOR = soup.find(
        "input", {"id": "__VIEWSTATEGENERATOR"}).attrs['value']
    EVENTVALIDATION = soup.find(
        "input", {"id": "__EVENTVALIDATION"}).attrs['value']

    # Retrieve the data required for selecting the Search options(filters)
    headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
               'Content-Type': 'application/x-www-form-urlencoded',
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    formfields = {'__VIEWSTATE': VIEWSTATE,
                  '__VIEWSTATEGENERATOR': VIEWSTATEGENERATOR,
                  '__EVENTVALIDATION': EVENTVALIDATION,
                  'ctl00$ContentPlaceHolder1$ddlQ_YM': month,
                  'ctl00$ContentPlaceHolder1$BtnQuery': '�윥屋�'}
    encodedFields = urlencode(formfields)

    # post request to 'aspx' page and get the data
    res = requests.post(url, data=encodedFields,headers=headers).text
    
    # for Open Interest
    # Only extract the column [0,1,2] and delete the 'first' and 'last' row, which are overlapping and unnecessary data
    if name == 'Open_Interest':
        df = pd.read_html(res)[4][[0, 1, 2]][1:-1]
        df.drop(df.index[0], inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.rename(columns={0: 'Fund_Name', 1: 'Contract_Month',
                           2: 'Open_Interest'}, inplace=True)
    # for Volume
    # Only extract the column [1,2,3,4,5] for Contract volume
    else:
        df = pd.read_html(res)[4][[1, 2, 3, 4, 5]][1:]
        df.drop(df.index[0], inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.rename(columns={1: 'Fund_Name', 2: 'Exchange',
                           3: 'Futures_Options_Contract', 4: month[4:]+'-\n'+'Buy', 5: month[4:]+'-\n'+'Sell'}, inplace=True)
    return df.copy()


def dateGenerator():
    # Function returns the list of date
    # Upward dependencies: None
    # Downward dependedncies: save_xlsx
    yearList = ['2019','2020']
    monthList=['01','02','03','04','05','06','07','08','09','10','11','12']
    #monthList = ['01', '02']
    Date = list()
    for year in yearList:
        for month in monthList:
            Date.append(year+month)
    Date.append('202101')
    return Date

def save_xlsx(url, soup, Date, name):
    # Function returns the whole (201901~202101) output excel file
    # Upward dependencies: checkServer
    # Downward dependedncies: None
    if name == 'Open_Interest':
        with ExcelWriter(f'TEX_{name}_202101.xlsx') as writer:
            for idx, date in enumerate(Date):
                print(f'Extracting {date} - {idx+1}/{len(Date)}')
                df = getMonthData(date, url, soup, name)
                df.to_excel(writer, f'{date}', index=False)
            writer.save()
    elif name =='Volume':
        with ExcelWriter(f'TEX_{name}_202101.xlsx') as writer:
            # sheet name format = [2019, 2020, 2021, ...]
            idx = 0
            for cnt in range((len(Date)-1)//12+1):
                while (idx < len(Date)):
                    date = Date[idx]
                    print(f'Extracting {date} - {idx+1}/{len(Date)}')
                    df = getMonthData(date, url, soup, name)
                    if idx%12 == 0:
                        merge_df = getMonthData(date, url, soup, name)
                    else:
                        merge_df = pd.merge(merge_df, df, on=['Fund_Name', 'Exchange', 'Futures_Options_Contract'], how='outer')
                        merge_df.drop_duplicates(inplace=True)
                    idx += 1
                    if (idx%12==0):
                        break
                # Fill 'nan' with '0', Drop Duplicates, Sort the data, Write in sheet
                merge_df.fillna('0', inplace=True)
                merge_df.drop_duplicates(inplace=True)
                merge_df = merge_df.sort_values(by=['Fund_Name', 'Exchange', 'Futures_Options_Contract'])
                merge_df.to_excel(writer, f'{date[:4]}', index=False)
            writer.save()

def addData(url, soup, Date, name):
    # Function addging the month data to existing file
    # Upward dependencies: checkServer
    # Downward dependedncies: None
    print(f'Extracting and Adding {Date} for {name}')
    
    if name=='Open_Interest':
        # if adding 'Jan'
        if Date[4:] == '01':
            # load the previous month file and create target month file
            book = load_workbook(f'TEX_{name}_{int(Date)-89}.xlsx')
            book.save(f'TEX_{name}_{Date}.xlsx')
        # if adding 'Feb' ~ 'Dec'
        else:
            # load the previous month file and create target month file
            book = load_workbook(f'TEX_{name}_{int(Date)-1}.xlsx')
            book.save(f'TEX_{name}_{Date}.xlsx')
            
        df = getMonthData(Date, url, soup, name)
        df.fillna('0', inplace=True)
        df.drop_duplicates(inplace=True)
        
    elif name=='Volume':
        # if adding 'January' data
        if Date[4:] == '01':
            # load the previous month file and create target month file
            book = load_workbook(f'TEX_{name}_{int(Date)-89}.xlsx')
            book.save(f'TEX_{name}_{Date}.xlsx')
            
            df = getMonthData(Date, url, soup, name)
            df.fillna('0', inplace=True)
            df.drop_duplicates(inplace=True)

        # if adding 'Feb' ~ 'Dec'
        else:
            # load the previous month file and create target month file
            book = load_workbook(f'TEX_{name}_{int(Date)-1}.xlsx')
            book.save(f'TEX_{name}_{Date}.xlsx')
    
            df_1 = pd.read_excel(f'TEX_{name}_{Date}.xlsx', index=False, sheet_name=f'{Date[:4]}', encoding='utf-8-sig')
            # if 'Date' already existing, drop the corresponding month column.
#             if f'{Date[4:]}-\nBuy' and f'{Date[4:]}-\nSell' in df_1.columns:
#                 df_1=df_1.drop([f'{Date[4:]}-\nBuy',f'{Date[4:]}-\nSell'], axis=1)
            
            df_2 = getMonthData(Date, url, soup, name)
            df = pd.merge(df_1, df_2, on=['Fund_Name', 'Exchange', 'Futures_Options_Contract'], how='outer')
            # Fill 'nan' with '0', Drop Duplicates, Sort the data, Write in sheet
            df.fillna('0', inplace=True)
            df.drop_duplicates(inplace=True)
            df = df.sort_values(by=['Fund_Name', 'Exchange', 'Futures_Options_Contract'])

    # overwriting existing sheet
    book = load_workbook(f'TEX_{name}_{Date}.xlsx')
    writer = pd.ExcelWriter(f'TEX_{name}_{Date}.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    if name=='Open_Interest':
        df.to_excel(writer, f"{Date}", index=False)
    elif name=='Volume':
        df.to_excel(writer, f"{Date[:4]}", index=False)
    writer.save()

start = datetime.now()

# Adding new month to existing file
if choice == 0:
    #OI
    soup = checkServer(url_1, 'Open_Interest')
    addData(url_1, soup, handle, 'Open_Interest')
    #Volume
    soup = checkServer(url_2, 'Volume')
    addData(url_2, soup, handle, 'Volume')
    
# Extracting whole data
elif choice == 1:
    Date = dateGenerator()
    #OI
    soup = checkServer(url_1, 'Open_Interest')
    save_xlsx(url_1, soup, Date, 'Open_Interest')
    #Volume
    soup = checkServer(url_2, 'Volume')
    save_xlsx(url_2, soup, Date, 'Volume')
    
print('Scraping Completed')
print(f'Total time taken: {datetime.now()-start} sec')

Open_Interest Server is up. Continue on scraping...
Extracting and Adding 202102 for Open_Interest
Volume Server is up. Continue on scraping...
Extracting and Adding 202102 for Volume
Scraping Completed
Total time taken: 0:00:22.306983 sec


In [12]:
name='Volume'
Date='202102'
book = load_workbook(f'TEX_{name}_{int(Date)-1}.xlsx')
book.save(f'TEX_{name}_{Date}.xlsx')

In [44]:
df.iloc[:,3:].sort_index(axis=1, inplace=True)
df.head()

,Fund_Name,Exchange,Futures_Options_Contract,01-\nBuy,01-\nSell
0,中國信託MSCI中國外資自由投資50不含A及B股單日正向2倍ETF基金,歐洲期貨交易 / EUX,MSCI 中國自由指數202103 / MSCI China Free Index 202103,69,88
1,保德信新興市場企業債券基金,芝加哥CBOT交易所 / CBOT,10年美債 / US 10YR NOTE,0,0
2,保德信新興市場企業債券基金,芝加哥CBOT交易所 / CBOT,2年美債 / US 2YR NOTE,0,0
3,保德信新興市場企業債券基金,芝加哥CBOT交易所 / CBOT,30年美債 / US Treasury Bond,0,0
4,保德信新興市場企業債券基金,芝加哥CBOT交易所 / CBOT,5年期美國公債 / US 5YR NOTE,0,0


In [7]:
from datetime import datetime
a = '202001'
b = '201912'
c = '202101'
d = '202012'
b = datetime.strptime(a, '%Y%m')
print(b)

2021-02-01 00:00:00
